## 작성한 KNN 알고리즘(함수) 설명

In [2]:
def my_knn(train, test, cl, k=1, method='Euclidean'):
    #사용되는 패키지 호출
    import numpy as np
    import pandas as pd
    #함수에 의한 입력데이터의 변형을 막기위해 복사하여 변수에 입력
    #들어오는 데이터의 형식에 상관없기위해 모두 넘파이로 변형
    train2 = np.array(train.copy())
    test2 = np.array(test.copy())
    cl2 = np.array(cl.copy())
    result = []
    for i in range(len(test)):
        #method 옵션에 따른 데이터별 거리계산
        if method=='Manhattan':
            temp = np.abs(train2 - test2[i])
        elif method=='Euclidean':
            temp = (train2 - test2[i])**2
        dist = np.sqrt(np.sum(temp, axis=1))
        #거리가 가까운 순으로 순위 입력
        rnk = pd.DataFrame({'rnk':dist}).rank(method='min')
        #정답 라벨(클래스) 추가
        rnk['cl'] = cl2
        #K 개수에 따른 정답 라벨 중 최고 빈도의 라벨을 추출
        top = rnk['cl'][rnk['rnk']<=k].mode()[0]
        #추출한 라벨을 결과 변수에 추가
        result.append(top)
    #결과를 나중에 비교분석(crosstab에 사용하기 위해 넘파이 형식으로 리턴)
    return np.array(result)

## my_knn으로 iris 데이터 분류

In [17]:
#데이터 준비
import  pandas  as  pd
iris= pd.read_csv("d:\\data\\iris.csv")
x = iris.iloc[:, :-1]
y = iris['variety']
from sklearn.preprocessing import  MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x)
x2 = scaler.transform(x)
y2 = y
from sklearn.model_selection import  train_test_split
x_train, x_test, y_train, y_test = train_test_split(x2, y2, test_size=0.2, random_state=0)

#분류 결과
print('#######유클리디안 거리#######')
result_uc = my_knn(x_train, x_test, y_train, 1)
print('정확도:', sum(result_uc == y_test)/len(y_test))
print(pd.crosstab(result_uc, y_test, rownames=['predicted'], colnames=['actual']), '\n')
print('#######맨해튼 거리#######')
result_mh = my_knn(x_train, x_test, y_train, 1, method='Manhattan')
print('정확도:', sum(result_mh == y_test)/len(y_test))
print(pd.crosstab(result_mh, y_test, rownames=['predicted'], colnames=['actual']))

#######유클리디안 거리#######
정확도: 0.9666666666666667
actual      Setosa  Versicolor  Virginica
predicted                                
Setosa          11           0          0
Versicolor       0          13          1
Virginica        0           0          5 

#######맨해튼 거리#######
정확도: 0.9333333333333333
actual      Setosa  Versicolor  Virginica
predicted                                
Setosa          11           0          0
Versicolor       0          12          1
Virginica        0           1          5


## 사이키런의 KNeighborsClassifier와 성능 비교

In [21]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors = 1)
model.fit(x_train, y_train)

print('#######사이키런#######')
result_sk = model.predict(x_test)
print('정확도:', sum(result_sk == y_test)/len(y_test))
print(pd.crosstab(result_sk, y_test, rownames=['predicted'], colnames=['actual']))

#######사이키런#######
정확도: 0.9666666666666667
actual      Setosa  Versicolor  Virginica
predicted                                
Setosa          11           0          0
Versicolor       0          13          1
Virginica        0           0          5
